#### 1. Открытие файла с данными, проверка пропусков и удаление строк с двумя и более пропусками

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import accuracy_score


data1 = pd.read_csv("penguins.csv")
data = pd.read_csv("penguins.csv")
print("Данные до обработки:\n", data.head())
missing_counts = data.isnull().sum()
print("\nКоличество пропусков в каждом столбце:\n", missing_counts)
data = data.dropna(thresh=data.shape[1] - 1)
print("\nДанные после удаления строк с >= 2 пропусками:\n", data.head())

Данные до обработки:
   species     island  bill_length_mm  bill_depth_mm  flipper_length_mm  \
0  Adelie  Torgersen            39.1           18.7              181.0   
1  Adelie  Torgersen            39.5           17.4              186.0   
2  Adelie  Torgersen            40.3           18.0              195.0   
3  Adelie  Torgersen             NaN            NaN                NaN   
4  Adelie  Torgersen            36.7           19.3              193.0   

   body_mass_g     sex  
0       3750.0    male  
1       3800.0  female  
2       3250.0  female  
3          NaN     NaN  
4       3450.0  female  

Количество пропусков в каждом столбце:
 species               0
island                0
bill_length_mm        2
bill_depth_mm         2
flipper_length_mm     2
body_mass_g           2
sex                  11
dtype: int64

Данные после удаления строк с >= 2 пропусками:
   species     island  bill_length_mm  bill_depth_mm  flipper_length_mm  \
0  Adelie  Torgersen            39.1  

#### 2. Разделение данных на data (без пропусков в 'sex') и data_with_nan (с пропусками в 'sex')

In [2]:
data_with_nan = data[data['sex'].isna()]
data = data.dropna(subset=['sex']) # - строки с пропусками в столбце

print("\nЧасть данных с пропусками в 'sex':\n", data_with_nan.head())
print("\nЧасть данных без пропусков в 'sex':\n", data.head())


Часть данных с пропусками в 'sex':
    species     island  bill_length_mm  bill_depth_mm  flipper_length_mm  \
8   Adelie  Torgersen            34.1           18.1              193.0   
9   Adelie  Torgersen            42.0           20.2              190.0   
10  Adelie  Torgersen            37.8           17.1              186.0   
11  Adelie  Torgersen            37.8           17.3              180.0   
47  Adelie      Dream            37.5           18.9              179.0   

    body_mass_g  sex  
8        3475.0  NaN  
9        4250.0  NaN  
10       3300.0  NaN  
11       3700.0  NaN  
47       2975.0  NaN  

Часть данных без пропусков в 'sex':
   species     island  bill_length_mm  bill_depth_mm  flipper_length_mm  \
0  Adelie  Torgersen            39.1           18.7              181.0   
1  Adelie  Torgersen            39.5           17.4              186.0   
2  Adelie  Torgersen            40.3           18.0              195.0   
4  Adelie  Torgersen            36.7    

#### 3. Решите задачу классификации пингвинов по полу методом KNN на наборе данных data:

###### отделите целевой признак от нецелевых
###### разделите данные на части для обучения train и тестирования test
###### проведите кодирование категориальных признаков в train, а затем в test используя кодировщики, обученные на train
###### проведите нормирование нецелевых признаков в train, а затем в test используя нормировщик, обученный на train
###### проведите подбор гиперпараметров (количество соседей и метрика расстояния) в KNN
###### обучите классификатор с наилучшими гиперпараметрами на train
###### проверьте его качество на test#### Блок 3: Разделение данных на части с пропусками и без пропусков в "sex"

In [3]:
# отделите целевой признак от нецелевых
X = data.drop(columns=['sex']) # остальные
y = data['sex'] # целевой

# разделите данные на части для обучения train и тестирования test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print("\nПример обучающих данных (X_train):\n", X_train.head())
print("\nПример целевого признака (y_train):\n", y_train.head())

# проведите кодирование категориальных признаков в train, а затем в test используя кодировщики, обученные на train
encoder = OneHotEncoder(handle_unknown='ignore') # кодирование категориальных признаков
X_train_encoded = encoder.fit_transform(X_train.select_dtypes(include=['object']))
X_test_encoded = encoder.transform(X_test.select_dtypes(include=['object']))


print("\nКатегориальные признаки после кодирования (train):\n", X_train_encoded.toarray()[:5])

# проведите нормирование нецелевых признаков в train, а затем в test используя нормировщик, обученный на train
scaler = StandardScaler() # нормализация числовых признаков
X_train_scaled = scaler.fit_transform(X_train.select_dtypes(exclude=['object']))
X_test_scaled = scaler.transform(X_test.select_dtypes(exclude=['object']))

X_train_final = np.hstack((X_train_scaled, X_train_encoded.toarray())) # объединение закодированных и нормализованных признаков
X_test_final = np.hstack((X_test_scaled, X_test_encoded.toarray()))
print("\nИтоговые признаки для обучения (train):\n", X_train_final[:5])

# проведите подбор гиперпараметров (количество соседей и метрика расстояния) в KNN
knn = KNeighborsClassifier() # подбор гиперпараметров для KNN
param_grid = {'n_neighbors': [3, 5, 7, 9], 'metric': ['euclidean', 'manhattan']}
grid_search = GridSearchCV(knn, param_grid, cv=5)
grid_search.fit(X_train_final, y_train)

# обучите классификатор с наилучшими гиперпараметрами на train
best_knn = grid_search.best_estimator_
print("\nЛучшие параметры для KNN:", grid_search.best_params_)

# проверьте его качество на test#### Блок 3: Разделение данных на части с пропусками и без пропусков в "sex"
y_pred = best_knn.predict(X_test_final) # тест на тестовых
accuracy = accuracy_score(y_test, y_pred)
print("\nТочность модели на тестовых данных:", accuracy)



Пример обучающих данных (X_train):
        species  island  bill_length_mm  bill_depth_mm  flipper_length_mm  \
232     Gentoo  Biscoe            49.1           14.5              212.0   
84      Adelie   Dream            37.3           17.8              191.0   
306  Chinstrap   Dream            40.9           16.6              187.0   
22      Adelie  Biscoe            35.9           19.2              189.0   
29      Adelie  Biscoe            40.5           18.9              180.0   

     body_mass_g  
232       4625.0  
84        3350.0  
306       3200.0  
22        3800.0  
29        3950.0  

Пример целевого признака (y_train):
 232    female
84     female
306    female
22     female
29       male
Name: sex, dtype: object

Категориальные признаки после кодирования (train):
 [[0. 0. 1. 1. 0. 0.]
 [1. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 1. 0.]
 [1. 0. 0. 1. 0. 0.]
 [1. 0. 0. 1. 0. 0.]]

Итоговые признаки для обучения (train):
 [[ 0.92804222 -1.30893172  0.74357505  0.48468373  0.     

#### 4. Используя обученный классификатор заполните пропуски пола в  data_with_nan. Предварительно надо

###### у data_with_nan  удалить столбец "sex"; 
###### провести кодирование нецелевых признаков обученным  на train кодировщиком; 
###### провести нормировку обученным на train нормализатором. 

In [4]:
X_nan = data_with_nan.drop(columns=['sex'])

X_nan_encoded = encoder.transform(X_nan.select_dtypes(include=['object'])) # кодирование категориальных

X_nan_scaled = scaler.transform(X_nan.select_dtypes(exclude=['object'])) # нормализация

X_nan_final = np.hstack((X_nan_scaled, X_nan_encoded.toarray()))
print("\nПризнаки для data_with_nan после кодирования и нормализации:\n", X_nan_final[:5])

sex_predicted = best_knn.predict(X_nan_final)
print("\nПредсказанные значения 'sex':", sex_predicted)

data_with_nan['sex'] = sex_predicted
print("\nОбновленные данные с предсказанным 'sex':\n", data_with_nan[['sex']])


Признаки для data_with_nan после кодирования и нормализации:
 [[-1.84461854  0.52276931 -0.59047831 -0.93581691  1.          0.
   0.          0.          0.          1.        ]
 [-0.38435054  1.59126157 -0.80111831  0.021477    1.          0.
   0.          0.          0.          1.        ]
 [-1.16069556  0.01396347 -1.08197165 -1.15198005  1.          0.
   0.          0.          0.          1.        ]
 [-1.16069556  0.11572464 -1.50325165 -0.65789287  1.          0.
   0.          0.          0.          1.        ]
 [-1.21614877  0.92981398 -1.57346499 -1.55342588  1.          0.
   0.          0.          1.          0.        ]]

Предсказанные значения 'sex': ['female' 'male' 'female' 'female' 'female' 'female' 'female' 'female'
 'male']

Обновленные данные с предсказанным 'sex':
         sex
8    female
9      male
10   female
11   female
47   female
178  female
218  female
256  female
268    male


##### 5. Объедините нецелевые столбцы из data_with_nan и предсказанные значения пола, выведите их. Сравните с заполнением, которое мы делали, ориентируясь на вес пингвина

In [5]:
final_data = pd.concat([data, data_with_nan], ignore_index=True)

print("\nОбъединенные данные:\n", final_data)
print("\nНачальные данные:\n", data1)
data1


Объединенные данные:
     species     island  bill_length_mm  bill_depth_mm  flipper_length_mm  \
0    Adelie  Torgersen            39.1           18.7              181.0   
1    Adelie  Torgersen            39.5           17.4              186.0   
2    Adelie  Torgersen            40.3           18.0              195.0   
3    Adelie  Torgersen            36.7           19.3              193.0   
4    Adelie  Torgersen            39.3           20.6              190.0   
..      ...        ...             ...            ...                ...   
337  Adelie      Dream            37.5           18.9              179.0   
338  Gentoo     Biscoe            44.5           14.3              216.0   
339  Gentoo     Biscoe            46.2           14.4              214.0   
340  Gentoo     Biscoe            47.3           13.8              216.0   
341  Gentoo     Biscoe            44.5           15.7              217.0   

     body_mass_g     sex  
0         3750.0    male  
1         

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,male
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,female
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,female
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,female
...,...,...,...,...,...,...,...
339,Chinstrap,Dream,55.8,19.8,207.0,4000.0,male
340,Chinstrap,Dream,43.5,18.1,202.0,3400.0,female
341,Chinstrap,Dream,49.6,18.2,193.0,3775.0,male
342,Chinstrap,Dream,50.8,19.0,210.0,4100.0,male


In [6]:
data1 = data1.dropna(thresh=data1.shape[1] - 1)

data_with_nan_initial = data1[data1['sex'].isna()]
print("\nСтроки с пропущенным полом из начального датафрейма:\n", data_with_nan_initial)

data_with_nan['sex'] = sex_predicted
print("\nОбновленные данные с предсказанным 'sex':\n", data_with_nan[['sex']])



Строки с пропущенным полом из начального датафрейма:
     species     island  bill_length_mm  bill_depth_mm  flipper_length_mm  \
8    Adelie  Torgersen            34.1           18.1              193.0   
9    Adelie  Torgersen            42.0           20.2              190.0   
10   Adelie  Torgersen            37.8           17.1              186.0   
11   Adelie  Torgersen            37.8           17.3              180.0   
47   Adelie      Dream            37.5           18.9              179.0   
178  Gentoo     Biscoe            44.5           14.3              216.0   
218  Gentoo     Biscoe            46.2           14.4              214.0   
256  Gentoo     Biscoe            47.3           13.8              216.0   
268  Gentoo     Biscoe            44.5           15.7              217.0   

     body_mass_g  sex  
8         3475.0  NaN  
9         4250.0  NaN  
10        3300.0  NaN  
11        3700.0  NaN  
47        2975.0  NaN  
178       4100.0  NaN  
218       4650.0